## Install & Import Standard Packages

In [2]:
#!pip install --upgrade pip
#!pip install matplotlib
#!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
#!pip install pandas

     - 2.8 MB 8.1 MB/s 0:00:000m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.5.22-py3-none-any.whl size=3005023 sha256=065f46a7495fbc417eaafcc5d2507cb19c3f4ed38ea3a138a6a9315cb282268a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n12nsiv1/wheels/4c/91/d1/c5369304e2f7afb660bb6eee093af5a7d3c0ea05a3c1e8c797
Successfully built yt-dlp


In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# https://pypi.org/project/yt-dlp/
import json
import yt_dlp

pd.set_option('display.max_columns', None)
import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')

## Import Playlist Data

In [7]:
#!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 2/2 [spotipy]



In [ ]:
# Download my daughter's playlist from Spotify

In [10]:
#https://developer.spotify.com/dashboard/7d18709cd0cd4c3594689384206d4ac6

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='###',
    client_secret='###',
    redirect_uri='http://127.0.0.1:8889/callback',
    scope="playlist-read-private"
))

#Emma's songs
playlist_id = '1eZmS24ui4yIpg0SKOs7oV' 
results = sp.playlist(playlist_id)

with open("emmas_playlist.json", "w") as f:
    json.dump(results, f, indent=2)

In [ ]:
# Extract the data necessary to add it to the other playlists

In [11]:
import json

# Load your downloaded JSON file
with open("emmas_playlist.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract general playlist info
playlist_info = {
    "name": data["name"],
    "pid": 42,  # You can change or generate this if needed
    "num_tracks": len(data["tracks"]["items"]),
    "num_albums": len(set(item["track"]["album"]["name"] for item in data["tracks"]["items"])),
    "num_artists": len(set(artist["name"]
                           for item in data["tracks"]["items"]
                           for artist in item["track"]["artists"])),
    "num_followers": data.get("followers", {}).get("total", 0),
    "collaborative": str(data.get("collaborative", False)).lower(),
    "0": []
}

# Add each track
for idx, item in enumerate(data["tracks"]["items"]):
    track = item["track"]
    playlist_info["0"].append({
        "track_name": track["name"],
        "pos": idx,
        "album_name": track["album"]["name"],
        "artist_name": track["artists"][0]["name"],  # First artist only
        "duration_ms": track["duration_ms"]
    })

# Save the cleaned playlist info
with open("cleaned_playlist.json", "w", encoding="utf-8") as f:
    json.dump(playlist_info, f, indent=2)


In [ ]:
# Get my playlist from YouTube

In [19]:
from yt_dlp import YoutubeDL
import json

ydl_opts = {
    'cookiefile': 'cookies.txt',
    'quiet': False,
    'skip_download': True,
    'extract_flat': False,
}

with YoutubeDL(ydl_opts) as ydl:
    url = "https://www.youtube.com/playlist?list=LL"
    try:
        info = ydl.extract_info(url, download=False)
    except Exception as e:
        print(f"❌ Failed to fetch playlist: {e}")
        exit(1)

    playlist_data = {
        "name": info.get("title", "Liked Videos"),
        "pid": info.get("id", "LL"),
        "num_tracks": 0,
        "num_albums": 0,
        "num_artists": 0,
        "num_followers": 0,
        "collaborative": "false",
        "0": []
    }

    seen_artists = set()
    successful_tracks = 0

    for i, entry in enumerate(info.get("entries", [])):
        try:
            if not entry:
                continue

            track_name = entry.get("title", "Unknown Title")
            artist_name = entry.get("uploader", "Unknown Artist")
            duration = entry.get("duration", 0)  # in seconds

            seen_artists.add(artist_name)

            track_info = {
                "track_name": track_name,
                "pos": successful_tracks,
                "album_name": "YouTube",
                "artist_name": artist_name,
                "duration_ms": duration * 1000
            }

            playlist_data["0"].append(track_info)
            successful_tracks += 1

        except Exception as e:
            print(f"⚠️ Skipping video at index {i}: {e}")
            continue

    # Finalize counts
    playlist_data["num_tracks"] = successful_tracks
    playlist_data["num_artists"] = len(seen_artists)

    # Save to JSON
    try:
        with open("liked_videos.json", "w", encoding="utf-8") as f:
            json.dump(playlist_data, f, indent=2, ensure_ascii=False)
        print(f"\n✅ Saved liked_videos.json with {successful_tracks} tracks.")
    except Exception as e:
        print(f"❌ Failed to save JSON: {e}")

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=LL
[youtube:tab] LL: Downloading webpage


ERROR: [youtube:tab] LL: YouTube said: The playlist does not exist.


❌ Failed to fetch playlist: ERROR: [youtube:tab] LL: YouTube said: The playlist does not exist.
⚠️ Skipping video at index 103: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 118: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 166: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 199: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 318: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 435: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 437: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 486: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 567: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at index 610: unsupported operand type(s) for *: 'NoneType' and 'int'
⚠️ Skipping video at ind

In [9]:
# Load the playlist
import json
import pandas as pd

# Load JSON from file
#working_dir = 'C:/Users/carja/Documents/Capstone/'
working_dir = '/mnt/c/Users/carja/Documents/Capstone/'
with open(working_dir + 'playlist_data3.json', encoding='utf-8') as f:
    playlists  = json.load(f)

# Flattened track list
track_rows = []

for playlist in playlists:
    # Extract playlist metadata (everything except the track list)
    playlist_metadata = {k: v for k, v in playlist.items() if k != '0'}
    
    # Extract the track list (the key "0" holds the tracks)
    tracks = playlist.get("0", [])
    
    for track in tracks:
        # Combine track data with playlist metadata
        combined = {**track, **playlist_metadata}
        track_rows.append(combined)

# Convert to DataFrame
df = pd.DataFrame(track_rows)

# Optional: Rename playlist fields to make it clear
playlist_fields = ['name', 'pid', 'num_tracks', 'num_albums', 'num_artists', 'num_followers', 'collaborative']
df.rename(columns={field: f'playlist_{field}' for field in playlist_fields}, inplace=True)

# first remove duplicate tracks to avoid downloading twice
df = df.drop_duplicates(subset=['artist_name', 'track_name'])

# Preview
df.head()

,track_name,pos,album_name,artist_name,duration_ms,playlist_name,playlist_pid,playlist_num_tracks,playlist_num_albums,playlist_num_artists,playlist_num_followers,playlist_collaborative
0,How I Feel,0,How I Feel,Flo Rida,169557,2014,8,21,21,19,1,false
1,Lo Que No Sabes Tú,1,Mi Niña Bonita - Reloaded,Chino & Nacho,232120,2014,8,21,21,19,1,false
2,Chucuchá,2,El Sonido,Ilegales,206959,2014,8,21,21,19,1,false
3,Mueva la colita,3,Top Caribe,Various Artists,242373,2014,8,21,21,19,1,false
4,Be My Baby,4,Lloviendo Estrellas,Leslie Grace,218253,2014,8,21,21,19,1,false


In [10]:
df.tail()

,track_name,pos,album_name,artist_name,duration_ms,playlist_name,playlist_pid,playlist_num_tracks,playlist_num_albums,playlist_num_artists,playlist_num_followers,playlist_collaborative
1538,The Green Man - Ragtime,717,YouTube,asx,376000,Liked videos,LL,729,0,530,0,false
1539,KIOKO et NISSUI YAKI ONIGIRI 6,720,YouTube,Kioko épicerie japonaise,166000,Liked videos,LL,729,0,530,0,false
1540,"""Weird Al"" Yankovic - Fat (Official Video)",723,YouTube,alyankovic,297000,Liked videos,LL,729,0,530,0,false
1541,Equestria Girls - Rainbow Rocks - 'My Past is ...,726,YouTube,My Little Pony - Official Channel,182000,Liked videos,LL,729,0,530,0,false
1542,Heartcatch Pretty Cure 3rd OP,727,YouTube,mintjn666,86000,Liked videos,LL,729,0,530,0,false


## EDA

### How many unique albums/tracks/artists are there on playlists?

In [8]:
# Unique albums
count = df[['artist_name', 'album_name']].drop_duplicates().shape[0]
print("Unique albums:", count)

# Unique tracks
count = df[['artist_name', 'track_name']].drop_duplicates().shape[0]
print("Unique tracks:", count)

# Unique artists
count = df[['artist_name']].drop_duplicates().shape[0]
print("Unique artists:", count)

Unique albums: 1146
Unique tracks: 1480
Unique artists: 929


### How many times is the next track from the same album/artist?

In [9]:
import pandas as pd

def count_sequential_matches(df, group_by='playlist_pid', order_by='pos', compare_field='artist_name'):
    """
    Count the number of times two consecutive tracks within the same playlist
    share the same value for a specified metadata field (e.g., artist, album).

    Parameters:
    - df: pandas.DataFrame with at least the playlist ID, track position, and metadata field.
    - group_by: column to group by (usually playlist ID).
    - order_by: column indicating order of tracks within group (usually track position).
    - compare_field: metadata field to compare between consecutive tracks.

    Returns:
    - count: int, number of sequential matches.
    """

    # Ensure correct ordering
    df_sorted = df.sort_values(by=[group_by, order_by]).copy()

    # Shift the compare_field to get previous track's value
    df_sorted['prev_value'] = df_sorted.groupby(group_by)[compare_field].shift()

    # Check for equality between current and previous track's metadata
    df_sorted['sequential_match'] = df_sorted[compare_field] == df_sorted['prev_value']

    # Count number of True values
    count = df_sorted['sequential_match'].sum()

    return count

count = count_sequential_matches(df, compare_field='album_name')
print("Consecutive tracks by the same album:", count)

count = count_sequential_matches(df)
print("Consecutive tracks by the same artist:", count)

Consecutive tracks by the same album: 786
Consecutive tracks by the same artist: 204


### Who are the top artists?

In [10]:
def get_top_artists(df, artist_col='artist_name', top_n=10):
    """
    Returns the top N artists based on track frequency in the dataset.

    Parameters:
    - df: pandas DataFrame containing tracks.
    - artist_col: column name containing artist names.
    - top_n: number of top artists to return.

    Returns:
    - pandas Series with artist names as index and counts as values.
    """
    return df[artist_col].value_counts().head(top_n)

top10Artists = get_top_artists(df)
print("Top 10 artists:", top10Artists)

Top 10 artists: artist_name
Eve                32
A.R. Rahman        26
RHINO              19
Lata Mangeshkar    19
natori             16
TopPop             15
Ed Sheeran         14
Arijit Singh       14
Ado                13
Pritam             13
Name: count, dtype: int64


### How many different albums are there for each artist?

In [11]:
album_counts = (
    df.groupby('artist_name')['album_name']
      .nunique()       # count unique albums per artist
      .reset_index()   # convert Series back to DataFrame
      .rename(columns={'album_name': 'unique_album_count'})
      .sort_values(by='unique_album_count', ascending=False)
      .head(10)
)

print("Top 10 artists by number of albums.\n\n", album_counts)

Top 10 artists by number of albums.

          artist_name  unique_album_count
234              Eve                  17
10       A.R. Rahman                  14
419  Lata Mangeshkar                  11
25               Ado                  11
896           natori                   9
420             Lavt                   8
55      Arijit Singh                   7
324     Jack Johnson                   7
735      The Beatles                   7
489    Mohammed Rafi                   7


# Audio Download
### https://github.com/yt-dlp/yt-dlp?tab=readme-ov-file#usage-and-options

In [42]:
#!pip install -U yt-dlp

Defaulting to user installation because normal site-packages is not writeable


'yt-dlp' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
import re

def sanitize_filename(filename):
    # Remove characters not allowed in filenames (Windows, macOS, Linux)
    return re.sub(r'[\\/*?:"<>|]', '', filename)

In [30]:
# Loop over your DataFrame rows and Download the music
import os

def download_track(search_query, output_template):
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': True,
        'noplaylist': True,
        'outtmpl': output_template,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f"ytsearch1:{search_query}"])
        #print(f"Downloaded: {search_query}")
    except Exception as e:
        print(f"Failed to download '{search_query}': {e}")   

# Make sure downloads folder exists
output_dir = working_dir + 'downloads'
os.makedirs(output_dir, exist_ok=True)

download_count = 0
skip_count, download_count = 0, 0
for ii, row in df.iterrows():
    artist = row['artist_name']
    track = row['track_name']
    safe_name = sanitize_filename(f"{artist} - {track}")
    output_template = output_dir + '/' + safe_name # os.path.join(output_dir, safe_name)

     # ✅ Skip if file already exists
    if os.path.exists(output_template):
        skip_count += 1
        print(f"Skipping (already exists): {output_template}")
        continue
    
    download_track(safe_name, output_template)
    download_count += 1

print(f"Skipped {skip_count}.  Downloaded {download_count}. Total {ii + 1}")

Skipped 0.  Downloaded 1480. Total 1543                                  


In [15]:
## Convert mp3 to wavfrom
import os
from pydub import AudioSegment

# Make sure downloads folder exists
wav_dir = working_dir + 'wavs'
os.makedirs(wav_dir, exist_ok=True)
mp3_dir = working_dir + 'downloads'

# Loop over your DataFrame rows
skip_count, wav_count = 0, 0
for idx, row in df.iterrows():
    artist = row['artist_name']
    track = row['track_name']
    mp3_name = sanitize_filename(f"{artist} - {track}.mp3")
    mp3_file = mp3_dir + '/' + mp3_name #os.path.join(output_dir, mp3_name)
    wav_name = sanitize_filename(f"{artist} - {track}.wav")
    wav_file = wav_dir + '/' + wav_name #os.path.join(wav_dir, wav_name)

     # ✅ Skip if file doesn't exists
    if not os.path.exists(mp3_file) or os.path.exists(wav_file):
        skip_count += 1
        continue
    wav_count += 1
    
    audio = AudioSegment.from_mp3(mp3_file)
    audio = audio.set_channels(1).set_frame_rate(44100)
    audio.export(wav_file, format="wav")

print(f"Skipped {skip_count}. Converted {wav_count}. Total {ii + 1}")

NameError: name 'ii' is not defined

In [ ]:
## Apply Signal Processing
### key signature, tempo, chroma, zero crossing rate, MFCC (Mel-frequency cepstral coefficients), spectral bandwidth, RMSE (Root Mean Square Error), etc
### I don't want to use a mood or genre classifier because it's a solved problem.
### I'm instead going to attempt to extract this metadata from the raw audio
### Then I'll calculate sonme distance score using all metadata and use it to match songs for recommendation 

In [14]:
#!pip install librosa
#!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [librosa]


In [16]:
### librosa: https://librosa.org/doc/latest/index.html
### (McFee, Brian, Colin Raffel, Dawen Liang, Daniel PW Ellis, Matt McVicar, Eric Battenberg, and Oriol Nieto.
### “librosa: Audio and music signal analysis in python.” In Proceedings of the 14th python in science conference, pp. 18-25. 2015.)

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

""" Degree of Harmony
    1.0–1.5 → monophonic (no harmony, single note per time).
    2.0–3.0 → likely vocal harmonies or chords (e.g., duets, backing vocals).
    3.0 → rich harmonic content (dense vocals, synths, or polyphonic instruments).
"""
def detectDegreeOfHarmony(chroma):
    # Define a threshold as a fraction of the maximum chroma energy
    threshold_ratio = 0.5
    counts_per_frame = []
    
    for frame in chroma.T:  # iterate over columns (time frames)
        threshold = threshold_ratio * np.max(frame)
        strong_pitches = np.sum(frame >= threshold)
        counts_per_frame.append(strong_pitches)
    
    # Convert to numpy array for easy stats
    counts_per_frame = np.array(counts_per_frame)
    
    average_polyphony = np.mean(counts_per_frame)
    return average_polyphony
    

# Krumhansl-Schmuckler key profiles
major_profile = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09,
                 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
minor_profile = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53,
                 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]

# Rotate profiles to each of the 12 keys
profiles = []
key_names = []

# Major keys
for i in range(12):
    profiles.append(np.roll(major_profile, i))
    key_names.append(librosa.midi_to_note(60 + i, octave=False) + ' major')

# Minor keys
for i in range(12):
    profiles.append(np.roll(minor_profile, i))
    key_names.append(librosa.midi_to_note(60 + i, octave=False) + ' minor')


# Loop over your DataFrame rows
skip_count, analyze_count = 0, 0
for ii, row in df.iterrows():
    artist = row['artist_name']
    track = row['track_name']
    safe_name = sanitize_filename(f"{artist} - {track}.mp3")
    song_file = output_dir + '/' + safe_name #os.path.join(output_dir, safe_name)
    #print("\nAnalyzing ", safe_name)

     # ✅ Skip if file doesn't exists
    if not os.path.exists(song_file):
        #print("Skipping ", song_file)
        skip_count += 1
        continue
    analyze_count += 1

    # Load the audio file
    y, sr = librosa.load(song_file)
    
    # Get the chroma features (energy per pitch class)
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    # Degree of Harmony
    df.at[ii, 'harmony_degree'] = detectDegreeOfHarmony(chroma)
    #print(f"Average number of strong pitch classes per frame: {df.at[ii, 'harmony_degree']:.2f}")

    # Average chroma over time to get a pitch class profile
    chroma_mean = np.mean(chroma, axis=1)
    # Correlate chroma vector with all key profiles
    correlations = [np.corrcoef(chroma_mean, profile)[0, 1] for profile in profiles]
    # Get the key with the highest correlation
    best_index = np.argmax(correlations)
    estimated_key = key_names[best_index]
    df.at[ii, 'estimated_key'] = estimated_key
    #print(f"Estimated key: {estimated_key}")
    
    
    # Detect tempo and beats
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    # Convert beat frames to time (seconds)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    tempo_val = float(np.squeeze(tempo))
    df.at[ii, 'tempo'] = tempo_val
    #print(f"Estimated tempo: {tempo_val:.2f} BPM")
    
    
    # MFCCs (timbre)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    df.at[ii, 'mfcc_shape'] = str(mfccs.shape) # convert Tuple to string to allow assignment
    #print("MFCC shape:", mfccs.shape)
    
    # Spectral features (reverb, distortion, timbre)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean, spectral_contrast_var = np.mean(spectral_contrast), np.std(spectral_contrast)
    df.at[ii, 'spectral_contrast_mean'] = spectral_contrast_mean
    df.at[ii, 'spectral_contrast_var'] = spectral_contrast_var
    #print(f"Average, Var spectral contrast: {spectral_contrast_mean:.2f}, {spectral_contrast_var:.2f} Hz")
    
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_centroid_mean, spectral_centroid_var = np.mean(spectral_centroid), np.std(spectral_centroid)
    df.at[ii, 'spectral_centroid_mean'] = spectral_centroid_mean
    df.at[ii, 'spectral_centroid_var'] = spectral_centroid_var
    #print(f"Average, Var spectral centroid: {spectral_centroid_mean:.2f}, {spectral_centroid_var:.2f} Hz")
    
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    spectral_flatness_mean, spectral_flatness_var = np.mean(spectral_flatness), np.std(spectral_flatness)
    df.at[ii, 'spectral_flatness_mean'] = spectral_flatness_mean
    df.at[ii, 'spectral_flatness_var'] = spectral_flatness_var
    #print(f"Average, Var spectral flatness: {spectral_flatness_mean:.2f}, {spectral_flatness_var:.2f} Hz")

print(f"Skipped {skip_count}.  Analyzed {analyze_count}. Total {ii + 1}")

## Save progress
df.to_csv(working_dir + 'initialAnalysis.csv', index=False)

NameError: name 'output_dir' is not defined

In [ ]:
## Load saved progress

In [48]:
#import pandas as pd
#df = pd.read_csv(working_dir + 'initialAnalysis.csv')

In [ ]:
## Identify Chord Progress
## Get Tonic from Key
## Mod based on Tonic
## Save "factorized" progression pattern

In [60]:
## Get Chord Progression & save to csv
import subprocess
import os

# Make sure chords folder exists
chords_dir = working_dir + 'chords'
os.makedirs(chords_dir, exist_ok=True)

# Loop over your DataFrame rows
skip_count, chord_count = 0, 0
for idx, row in df.iterrows():
    artist = row['artist_name']
    track = row['track_name']
    wav_name = sanitize_filename(f"{artist} - {track}.wav")
    wav_file = wav_dir + '/' + wav_name #os.path.join(wav_dir, wav_name)
    chords_name = sanitize_filename(f"{artist} - {track}.csv")
    chords_file = chords_dir + '/' + chords_name # os.path.join(chords_dir, chords_name)
    
     # ✅ Skip if file doesn't exists
    if not os.path.exists(wav_file) or (os.path.exists(chords_file) and os.path.getsize(chords_file) > 0):
        skip_count += 1
        continue
    chord_count += 1
    
    command = [
        'C:/sonic-annotator-1.6-win64/sonic-annotator.exe',
        '-d', 'vamp:nnls-chroma:chordino',
        '-w', 'csv',
        '--csv-stdout',
        wav_file
    ]
    
    # Run and capture output
    with open(chords_file, "w") as f:
        subprocess.run(command, stdout=f, shell=True)
        #result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        #print("STDOUT:", result.stdout.decode())
        #print("STDERR:", result.stderr.decode())

print(f"Skipped {skip_count}. Converted {chord_count}. Total {ii + 1}")

Skipped 730. Converted 1. Total 731


In [30]:
## use the chord output to factorize to an estimate per track chord progression

In [45]:
!pip install music21==6.1.0 --no-build-isolation

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
     ------ --------------------------------- 2.9/18.3 MB 16.3 MB/s eta 0:00:01
     ----------- ---------------------------- 5.2/18.3 MB 15.6 MB/s eta 0:00:01
     ------------------ --------------------- 8.4/18.3 MB 15.1 MB/s eta 0:00:01
     ------------------------ -------------- 11.5/18.3 MB 14.8 MB/s eta 0:00:01
     -------------------------- ------------ 12.6/18.3 MB 12.6 MB/s eta 0:00:01
     ------------------------------ -------- 14.4/18.3 MB 11.8 MB/s eta 0:00:01
     ----------------------------------- --- 16.8/18.3 MB 12.2 MB/s eta 0:00:01
     --------------------------------------  17.8/18.3 MB 11.1 MB/s eta 0:00:01
     --------------------------------------- 18.3/18.3 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached c

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [191]:
from music21 import key as m21key, roman, chord, pitch, harmony
from collections import Counter
import re
from typing import List, Tuple, Optional
from collections import defaultdict

# Simplify chords
def clean_chord(ch):
    # Only use base chord (before colon) and strip whitespace
    return ch.split(":")[0].strip()

def normalize_chord_symbol(ch_str):
    # Split bass note off
    base = ch_str.split('/')[0]
    
    # Replace unicode flats '♭' with 'b'
    base = base.replace("♭", "b")
    # Replace flats 'b' that occur immediately after a note letter with '-'
    # e.g. 'Eb7' -> 'E-7'
    import re
    # Pattern: letter followed by 'b'
    base = re.sub(r'([A-Ga-g])b', r'\1-', base)
    
    # Replace unicode sharps '♯' with '#'
    base = base.replace('♯', '#')
    
    # Recombine with bass if present
    if '/' in ch_str:
        bass = ch_str.split('/')[1]
        return base + '/' + bass
    else:
        return base

def simplify_to_triad(chord_str: str):
    try:
        chord_str.closedPosition(forceOctave=4, inPlace=True)
        root = chord_str.root()
        quality = chord_str.commonName

        # Decide triad type based on commonName
        if "major" in quality.lower():
            return harmony.ChordSymbol(f"{root.name}")
        elif "minor" in quality.lower():
            return harmony.ChordSymbol(f"{root.name}m")
        elif "diminished" in quality.lower() or "dim" in quality.lower():
            return harmony.ChordSymbol(f"{root.name}dim")
        elif "augmented" in quality.lower() or "aug" in quality.lower():
            return harmony.ChordSymbol(f"{root.name}aug")
        else:
            return harmony.ChordSymbol(root.name)  # Default to major if unclear
    except Exception as e:
        print(f"Failed to simplify {chord_str}: {e}")
        return None

'''
    Map chords to Roman numerals 
    We first simplify the chords to their triads
    This makes pattern recognition stronger by de-emphasizing embelishments
'''
def chord_to_roman(chord_symbol, song_key):
        try:
            # Try parsing with ChordSymbol (handles complex symbols better)
            cs = harmony.ChordSymbol(chord_symbol)
            # This sometimes returns no pitches if parsing failed, check:
            if not cs.pitches:
                raise ValueError(f"No pitches parsed for chord '{chord_symbol}'")
            triad = simplify_to_triad(cs)
            rn = roman.romanNumeralFromChord(triad, song_key)
            return rn.figure
        except Exception as e:
            # As fallback, try parsing just root note and quality manually
            try:
                # Remove bass note (after slash)
                base_chord = chord_symbol.split('/')[0]
                cs = harmony.ChordSymbol(base_chord)
                triad = simplify_to_triad(cs)
                rn = roman.romanNumeralFromChord(triad, song_key)
                return rn.figure
            except Exception as e2:
                print(f"Failed to convert '{chord_symbol}': {e2}")
                return None

'''
    Searches the sequence for looped sequences. if none is found returns the most frequent patterns.
    weights resolutions that resolve to the key
    weights length to prevent "greedy" tendency of shorter sequences to dominate repetitions
    reduces overlap in sequences by checking equality, subset, and intersection > min_len
'''
def is_subpattern(smaller, larger):
    """Check if 'smaller' is a contiguous subpattern of 'larger'."""
    for i in range(len(larger) - len(smaller) + 1):
        if tuple(larger[i:i + len(smaller)]) == smaller:
            return True
    return False

def cadence_bonus(pattern):
    """Add score based on how the pattern ends."""
    if len(pattern) < 2:
        return 0

    end = pattern[-2:]
    last = pattern[-1]

    # Strong cadences
    if end == ['V', 'I'] or end == ['V7', 'I']:
        return 8  # Authentic cadence
    if end == ['IV', 'I']:
        return 5  # Plagal cadence
    if end == ['V', 'vi']:
        return 3  # Deceptive cadence
    if pattern[-3:] == ['ii', 'V', 'I']:
        return 6  # 2-5-1 cadence
    if pattern[-3:] == ['I', 'IV', 'V'] or pattern[-4:] == ['I', 'IV', 'V', 'I']:
        return 6  # Full cadence
    if pattern[-4:] == ['I', 'V', 'vi', 'IV']:
        return 4  # Axis progression
    if end == ['bVII', 'I']:
        return 4  # Modal cadence (Mixolydian)

    # Melodic feel resolutions
    if last == 'I':
        return 2  # Tonic resolution
    if last == 'iii':
        return 2  # Mediant (3rd) resolution
    if last == 'vi':
        return 1  # Submediant resolution

    return 0



def find_top_two_progression_patterns(progression, min_len, max_len):
    n = len(progression)
    pattern_counts = defaultdict(int)
    pattern_positions = defaultdict(list)

    # Step 1: Identify all patterns and their positions
    for i in range(n):
        for length in range(min_len, max_len + 1):
            if i + length <= n:
                pattern = tuple(progression[i:i + length])
                pattern_counts[pattern] += 1
                pattern_positions[pattern].append((i, i + length))  # store as (start, end)

    scored_patterns = []

    for pattern, positions in pattern_positions.items():
        last_chord = pattern[-1]
        resolution_bonus = cadence_bonus(pattern)

        # Check for strict looping
        is_looping = False
        if len(positions) > 1:
            distances = [j[0] - i[0] for i, j in zip(positions, positions[1:])]
            if all(d == len(pattern) for d in distances):
                is_looping = True

        base_score = len(pattern) + resolution_bonus if is_looping else pattern_counts[pattern] * len(pattern) + resolution_bonus
        scored_patterns.append((base_score, pattern, pattern_counts[pattern], positions))

    # Sort by score descending
    scored_patterns.sort(key=lambda x: x[0], reverse=True)

    top_matches = []

    for score, pattern, count, positions in scored_patterns:
        pattern_list = list(pattern)

        # Check against previously selected patterns
        is_valid = True
        for existing in top_matches:
            # Subset/duplicate check
            if (
                pattern == tuple(existing['pattern']) or
                is_subpattern(pattern, existing['pattern']) or
                is_subpattern(existing['pattern'], pattern)
            ):
                is_valid = False
                break

            # Overlap check
            for p_start, p_end in positions:
                for e_start, e_end in existing['positions']:
                    overlap = max(0, min(p_end, e_end) - max(p_start, e_start))
                    if overlap > min_len:
                        is_valid = False
                        break
                if not is_valid:
                    break

        if is_valid:
            top_matches.append({
                "pattern": pattern_list,
                "repetitions": count,
                "positions": positions
            })

        if len(top_matches) == 2:
            break

    # Remove 'positions' before returning if not needed
    for match in top_matches:
        del match['positions']

    return top_matches


# Loop over your DataFrame rows
skip_count, progression_count = 0, 0
for ii, row in df.iterrows():
    artist = row['artist_name']
    track = row['track_name']
    chords_name = sanitize_filename(f"{artist} - {track}.csv")
    chords_file = chords_dir + '/' + chords_name # os.path.join(chords_dir, chords_name)
    
     # ✅ Skip if file doesn't exists
    if not os.path.exists(chords_file) or os.path.getsize(chords_file) <= 0:
        skip_count += 1
        continue
    progression_count += 1
    
    # Step 1: Define key
    key_str = row["estimated_key"].replace("♯", "#").replace("♭", "b")  # Change this to your key
    tonic, mode = key_str.split()
    df.at[ii, 'tonic'] = tonic
    df.at[ii, 'mode'] = mode
    song_key = m21key.Key(tonic, mode)

    # sanitze chords and translate to sequences within their key
    df_chord = pd.read_csv(chords_file, comment="#", header=None)
    df_chord.columns = ["start", "duration", "chord"]

    # clean chord syntax, simplify to their closest triad
    # mod to their key to represent as general numeric progression for key-indifferent comparison
    chords_clean = [clean_chord(ch) for ch in df_chord["chord"] if ch != "N"]
    normal_chords = [normalize_chord_symbol(ch) for ch in chords_clean if ch != "N"]
    roman_chords = [chord_to_roman(ch, song_key) for ch in normal_chords]
    roman_chords = [rc for rc in roman_chords if rc is not None]

    # most songs are in 4/4 time so we'll look for sequences divisible by 4
    sequences = find_top_two_progression_patterns(roman_chords, 4, 8)
    #print(f"\n{chords_name}")
    
    for jj, seq in enumerate(sequences, 1):
        #print(f"Progressions #{jj}: Pattern = {seq['pattern']}, Repetitions = {seq['repetitions']}")
        df.at[ii, 'progression' + str(jj)] = seq['pattern']

print(df.head())
print(f"Skipped {skip_count}. Analyzed {progression_count}. Total {ii + 1}")

           track_name  pos                 album_name      artist_name  \
0          How I Feel    0                 How I Feel         Flo Rida   
1  Lo Que No Sabes Tú    1  Mi Niña Bonita - Reloaded    Chino & Nacho   
2            Chucuchá    2                  El Sonido         Ilegales   
3     Mueva la colita    3                 Top Caribe  Various Artists   
4          Be My Baby    4        Lloviendo Estrellas     Leslie Grace   

   duration_ms playlist_name  playlist_pid  playlist_num_tracks  \
0       169557          2014             8                   21   
1       232120          2014             8                   21   
2       206959          2014             8                   21   
3       242373          2014             8                   21   
4       218253          2014             8                   21   

   playlist_num_albums  playlist_num_artists  playlist_num_followers  \
0                   21                    19                       1   
1       

In [192]:
#save progress
df.to_csv(working_dir + 'analyzedTracks.csv', index=False)